In [ ]:
def parse_time_to_seconds(time_str):
    h_str, m_str, s_str = time_str.split(':')
    h = int(h_str.rstrip('h'))
    m = int(m_str.rstrip('m'))
    s = int(s_str.rstrip('s'))
    return h * 3600 + m * 60 + s

def format_mmss(total_seconds):
    minutes = total_seconds // 60
    seconds = total_seconds % 60
    return f"{minutes:02d}m:{seconds:02d}s"

res = []
for model_name, times in timings.items():
    feat_sec = parse_time_to_seconds(times["feature_generation"])
    hp_sec = parse_time_to_seconds(times["hp_tuning"])
    train_sec = parse_time_to_seconds(times["training"])

    data_proc = feat_sec
    tuning = hp_sec - feat_sec
    training = train_sec - hp_sec
    total = train_sec

    res.append({
        "Model": model_name,
        "Data Processing": format_mmss(data_proc),
        "Tuning": format_mmss(tuning),
        "Training": format_mmss(training),
        'Total': format_mmss(total)
    })

df_timings = pd.DataFrame(res).set_index("Model")
df_timings

# MAE and RMSE on hourly corrected data

In [ ]:
import pandas as pd

# load forecasts
forecasts = pd.read_csv('../data/processed/forecasts.csv', parse_dates=True).dropna()
df_price = pd.read_csv('../data/raw/de_prices_2024.csv', parse_dates=True, skiprows=2, index_col=0, names=['de_prices'], dtype={'de_prices': float})
df_price.index = df_price.index.tz_localize(None)

forecasts.set_index('target_timestamp', inplace=True)
forecasts.index = pd.to_datetime(forecasts.index)

start = forecasts.index[0]
end = forecasts.index[-1]

df_price = df_price.loc[start:end]

true_prices = df_price.loc[: ,['de_prices']]

In [ ]:
errors = pd.DataFrame()

for col in forecasts.columns:
    errors[col + '_error'] = forecasts[col] - true_prices['de_prices']

In [ ]:
import pickle as pkl
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt

with open("../data/processed/performance.pkl", "rb") as f:
    performance = pkl.load(f)

drop_cols = [col for col in performance.keys() if not col.startswith('shifted')]
p = pd.DataFrame(performance).drop(columns=drop_cols).T.sort_values('mean_absolute_error', ascending=False)

plt.figure(figsize=(12, 4))

sns.barplot(data=p, x=p.index, y='mean_absolute_error')
plt.xticks(rotation=90)
plt.show()